# Avaliações de Vinhos

Abaixo podemos ver dados sobre avaliações de vinhos pelo mundo em comparação com o Brasil.

In [26]:
import pandas as pd

df = pd.read_csv(r'..\\data\\interim\\tech_challenge\\comercio_vinhos_rs.csv', sep=';', decimal=',')

# Remove as linhas que não contêm "_" na coluna "id_produto"
df_filtered = df[df['id_produto'].str.contains('_')]

# Exibe apenas o cabeçalho do DataFrame filtrado
display(df_filtered.head())

,id_produto,produto,ano,quantidade_com_rs
1,vm_tinto,tinto,1970,83300735
2,vm_rosado,rosado,1970,107681
3,vm_branco,branco,1970,14919190
5,vm_tinto,tinto,1970,435354
6,vm_rosado,rosado,1970,183234


In [27]:
# Identifica o maior ano presente na coluna "ano"
maior_ano = df_filtered['ano'].max()

# Calcula o limite inferior do intervalo de 15 anos
limite_inferior = maior_ano - 15

# Filtra o DataFrame para exibir apenas as linhas a partir de 15 anos em relação ao maior ano
df_final = df_filtered[df_filtered['ano'] >= limite_inferior]

display(df_final.head())

,id_produto,produto,ano,quantidade_com_rs
2197,vm_tinto,tinto,2006,209438207
2198,vm_rosado,rosado,2006,3071382
2199,vm_branco,branco,2006,33116025
2201,vm_tinto,tinto,2006,13920655
2202,vm_rosado,rosado,2006,307189


In [28]:
# Agrupa por produto e calcula a soma da quantidade exportada
df_soma_exportacao = df_final.groupby('produto')['quantidade_com_rs'].sum().reset_index()

# Calcula a soma total
soma_total = df_soma_exportacao['quantidade_com_rs'].sum()

# Calcula a representatividade da quantidade
df_soma_exportacao['representatividade'] = df_soma_exportacao['quantidade_com_rs'] / soma_total * 100

# Ordena o DataFrame pela soma total em ordem decrescente
df_soma_exportacao = df_soma_exportacao.sort_values(by='quantidade_com_rs', ascending=False)

# Exibe o DataFrame com a soma total exportada agrupada por produto, ordenado e com a representatividade da quantidade
display(df_soma_exportacao.head(10))

,produto,quantidade_com_rs,representatividade
36,tinto,3106803887,63.426905
10,branco,533300132,10.887580
33,suco natural integral,361256175,7.375220
15,espumante,181997854,3.715574
18,filtrado doce,90392457,1.845406
39,vinagre simples,86216069,1.760143
16,espumante moscatel,77377936,1.579708
12,cooler,68158121,1.391481
24,nectar de uva,60274382,1.230531
35,suco reprocessado/reconstituido,55962763,1.142507


In [29]:
df = pd.read_csv(r'..\\data\\processed\\wine_ratings\\winemag-data-130k-v2.csv', delimiter=',')

# display(df)

In [30]:
filtered_df = df[['country', 'designation', 'points', 'province', 'title', 'variety']]

# display(filtered_df['variety'].unique())

In [40]:
import ..reports.config as config
filtered_df['tipo'] = filtered_df['variety'].map(lambda x: tipo_vinho_dict.get(x, 'Outro'))

# Remover as linhas em que o tipo de vinho é 'Outro'
filtered_df = filtered_df[filtered_df['tipo'] != 'Outro']

# Exibir o DataFrame atualizado
display(filtered_df)


ImportError: attempted relative import with no known parent package

In [32]:
# Agrupar os dados por país e tipo, calculando a média dos pontos
grouped_df = filtered_df.groupby(['country', 'tipo']).agg({'points': 'mean'})

# Resetar o índice do DataFrame
grouped_df = grouped_df.reset_index()

# Ordenar o DataFrame agrupado por país em ordem alfabética
grouped_df_sorted = grouped_df.sort_values(by='country')

# Exibir o DataFrame ordenado
display(grouped_df_sorted)


,country,tipo,points
0,Argentina,branco,84.743590
1,Argentina,espumante,84.729730
2,Argentina,rosado,84.131579
3,Argentina,tinto,87.206461
4,Armenia,rosado,88.000000
...,...,...,...
119,Ukraine,branco,86.500000
122,Ukraine,tinto,84.000000
124,Uruguay,rosado,84.333333
123,Uruguay,branco,82.800000


In [33]:
grouped_df_sorted[(grouped_df_sorted['country'] == 'Brazil')]

,country,tipo,points
14,Brazil,espumante,85.769231
15,Brazil,tinto,84.368421
13,Brazil,branco,83.800000


In [37]:
df['tipo'] = df['variety'].map(lambda x: tipo_vinho_dict.get(x, 'Outro'))
df[(df['country'] == 'Brazil')][['tipo','variety']].drop_duplicates()

,tipo,variety
5852,espumante,Sparkling Blend
10565,Outro,Tannat
10579,tinto,Cabernet Franc
12669,Outro,Cabernet Sauvignon-Merlot
13218,espumante,Champagne Blend
14726,tinto,Bordeaux-style Red Blend
14734,tinto,Cabernet Sauvignon
14739,branco,Chardonnay
15989,Outro,Moscatel
25081,tinto,Pinot Noir


In [11]:
import plotly.graph_objects as go

# Ordenar o DataFrame por tipo, média e país, com points em ordem decrescente
grouped_df = grouped_df.sort_values(by=['tipo', 'points', 'country'], ascending=[True, False, True])

# Criar os gráficos de barra separados
figs = []

# Definir as cores para cada tipo de vinho
colors = {'tinto': 'red', 'branco': 'grey', 'rosado': 'pink'}

# Encontrar a pontuação média do país "Brazil"
brazil_mean_points = grouped_df[(grouped_df['country'] == 'Brazil')]['points'].mean()

# Gerar um gráfico de barra para cada tipo de vinho
for tipo in grouped_df['tipo'].unique():
    data = grouped_df[grouped_df['tipo'] == tipo]
    data = data.sort_values(by='points', ascending=False)

    # Criar a figura para o tipo de vinho atual
    fig = go.Figure()

    # Adicionar um trace de barra para o tipo de vinho atual
    fig.add_trace(go.Bar(x=data['country'], y=data['points'], name=tipo, marker_color=colors[tipo]))

    # Adicionar uma linha horizontal tracejada no valor da pontuação média do país "Brazil"
    fig.add_shape(
        type='line',
        x0=data['country'].iloc[0],
        x1=data['country'].iloc[-1],
        y0=brazil_mean_points,
        y1=brazil_mean_points,
        line=dict(color='black', width=3, dash='dash')
    )

    # Configurar o layout do gráfico atual
    fig.update_layout(
        title=f'Média de Pontuação por País - Tipo: {tipo}',
        xaxis=dict(title='País'),
        yaxis=dict(title='Pontuação Média'),
        barmode='group'
    )

    figs.append(fig)

# Exibir os gráficos separadamente, um abaixo do outro
for fig in figs:
    fig.show()

In [12]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Histogram(x=filtered_df['points'], nbinsx=20)])

fig.update_layout(
    xaxis_title='Pontos',
    yaxis_title='Frequência',
    title='Distribuição dos Pontos dos Vinhos'
)

fig.show()


In [13]:
# Filtrar apenas as colunas desejadas
filtered_df = df[['country', 'variety', 'points']]

# Agrupar por país e variedade de uva e calcular as estatísticas
statistics_by_country_variety = filtered_df.groupby(['country', 'variety'])['points'].agg(['mean', 'median', 'min', 'max', 'std', 'count'])

# Filtrar combinações de país e variedade com pelo menos 30 avaliações
filtered_countries_varieties = statistics_by_country_variety[statistics_by_country_variety['count'] >= 30]

# Ordenar as combinações de país e variedade em ordem decrescente da maior média de pontos
sorted_countries_varieties = filtered_countries_varieties.sort_values(by='mean', ascending=False)

# Exibir as combinações de país e variedade ordenadas com pelo menos 30 avaliações
print(sorted_countries_varieties)

                                mean  median  min  max       std  count
country   variety                                                      
France    Petit Manseng    91.944444    92.0   86   95  1.739367     36
          Tannat           91.508475    91.0   86   96  2.200101     59
Austria   Riesling         91.428571    92.0   84   96  2.435473    581
England   Sparkling Blend  91.340426    91.0   89   95  1.821091     47
Spain     Pedro Ximénez    90.857143    91.0   87   96  2.102420     35
...                              ...     ...  ...  ...       ...    ...
Portugal  Rosé             84.557447    84.0   80   90  1.476272    235
Spain     Sauvignon Blanc  84.363636    85.0   80   88  1.997157     33
Argentina Sauvignon Blanc  84.346154    84.0   80   89  1.885191     78
Chile     Rosé             84.250000    84.0   80   88  1.869274     44
Argentina Rosé             84.131579    84.5   80   87  2.170702     38

[292 rows x 6 columns]


In [14]:
import pandas as pd
import plotly.graph_objects as go

# Filtrar apenas as colunas desejadas
filtered_df = df[['country', 'points']]

# Agrupar por país e calcular as estatísticas
statistics_by_country = filtered_df.groupby('country')['points'].agg(['mean', 'median', 'min', 'max', 'std', 'count'])

# Filtrar países com pelo menos 30 avaliações
filtered_countries = statistics_by_country[statistics_by_country['count'] >= 30]

# Selecionar os top 10 países com base na média de pontos
top_10_countries = filtered_countries.nlargest(10, 'mean')

# Configurar os dados para o gráfico
mean_points = top_10_countries['mean']
countries = top_10_countries.index

# Criar o gráfico de candlesticks com a média no eixo esquerdo e o país no eixo direito
fig = go.Figure()

# Configurar os valores para o gráfico de candlesticks
candlestick_data = []
for country in top_10_countries.index:
    country_data = filtered_df[filtered_df['country'] == country]['points']
    candlestick_data.append([country_data.min(), country_data.max(), country_data.mean()])

# Plotar o gráfico de candlesticks
candlestick_colors = ['red', 'green', 'blue']
candlestick_labels = ['Min', 'Max', 'Mean']

for i, (min_val, max_val, median_val) in enumerate(candlestick_data):
    fig.add_trace(go.Candlestick(x=[countries[i]], open=[min_val], high=[max_val], low=[min_val], close=[max_val], name=countries[i], increasing_line_color='green', decreasing_line_color='red'))
    fig.add_trace(go.Scatter(x=[countries[i]], y=[median_val], mode='markers', name='Mean', marker=dict(color='blue', size=8)))

# Configurar o layout do gráfico
fig.update_layout(
    title='Candlestick Plot - Top 10 Countries',
    xaxis=dict(rangeslider=dict(visible=False)),
    yaxis=dict(title='Wine Ratings'),
    showlegend=False
)

# Exibir o gráfico
fig.show()